# Install dependencies
See `install_dependencies.ipynb`.

# Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from matplotlib import pyplot as plt
import IPython
import cv2
import itertools
import numpy as np
import pandas as pd
import grpc
import json
import base64
import datetime
import time
import imp

In [ ]:
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream

In [ ]:
imp.reload(video);

In [ ]:
pd.set_option('display.max_colwidth', -1)

# Define Pravega stream parameters

In [ ]:
gateway = 'hop-claudio-ubuntu18-desktop.solarch.lab.emc.com:54672'
scope = 'examples'
stream = 'object-detector-input-video'

# Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

# Copy from a video file to Pravega

In [ ]:
filename = '../../data/SampleVideo_1280x720_5mb.mp4'
output_stream = OutputStream(pravega_client, scope, stream)

In [ ]:
%%time
output_stream.write_video_from_file(filename, crop=[int(c) for c in [1280*.25,720*.25,1280*.75,720*.75]])

# Get head and tail stream cuts

In [ ]:
unindexed_stream = UnindexedStream(pravega_client, scope, stream)
stream_info = unindexed_stream.get_stream_info()
stream_info

In [ ]:
stream_size_MB = (stream_info.tail_stream_cut.cut[0] - stream_info.head_stream_cut.cut[0]) * 1e-6
stream_size_MB